# Modelado de Machine Learning para Campañas de Marketing Bancario

## Objetivo

El objetivo de este notebook es entrenar y evaluar modelos de clasificación binaria para estimar la probabilidad de que un cliente acepte una campaña de marketing bancario (y = yes/no).

El modelo está pensado como una herramienta de priorización de clientes, permitiendo enfocar los esfuerzos de contacto en aquellos con mayor propensión a aceptar la campaña, y no como un sistema de predicción perfecta.

Dado el desbalance de clases presente en el dataset (aproximadamente 11% de respuestas positivas), se prioriza el recall de la clase positiva, buscando maximizar la identificación de clientes potencialmente interesados, aun a costa de un mayor número de falsos positivos.

Este análisis se enfoca en modelos interpretables y realistas para un contexto productivo, evitando técnicas excesivamente complejas o difíciles de explicar a áreas de negocio.

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    RocCurveDisplay
)


## Carga de datos

In [12]:
df = pd.read_csv(r"C:\Users\Julio\Documents\Python\Datos_limpios_BM.csv", delimiter=',')

In [13]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,y_encoded,contact_frequency,age_group
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,1,Adulto
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,1,Adulto
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,1,Adulto joven
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,1,Adulto joven
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0,1,Adulto


In [14]:
df.shape

(41176, 24)

In [15]:
RANDOM_STATE = 42

El dataset utilizado ya fue analizado y validado en un notebook previo de EDA, donde se verificó la ausencia de valores nulos críticos, duplicados relevantes y problemas de calidad de datos.

In [16]:
y = df['y'].map({'yes': 1, 'no': 0})
y.value_counts(normalize=True)

y
0    0.887337
1    0.112663
Name: proportion, dtype: float64

La variable objetivo se define como una clasificación binaria, donde 1 representa la aceptación de la campaña. Dado que la proporción de respuestas positivas es significativamente menor, el problema presenta un desbalance de clases que influye en la elección de métricas y en el entrenamiento de los modelos.

In [17]:
X = df.drop(columns='y')

## Selección y justificación de variables

In [18]:
X = X.drop(columns=['duration'])

In [19]:
X = X.drop(columns=['y_encoded'])

In [20]:
X = X.drop(columns=['age_group'])

In [21]:
X = X.drop(columns=['contact_frequency'])

In [22]:
numeric_features = [
    'age',
    'campaign',
    'pdays',
    'previous',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed'
]


In [23]:
categorical_features = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'day_of_week',
    'poutcome'
]

In [24]:
set(numeric_features + categorical_features) == set(X.columns)

True

Excluí variables derivadas del EDA para evitar redundancia y priorizar simplicidad. También eliminé `duration` por leakege, ya que no estaría disponible antes de la llamada.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
X,
y,
test_size=0.2,
stratify=y,
random_state=RANDOM_STATE
)

In [26]:
categorical_transformer = OneHotEncoder(
    drop='first',
    handle_unknown='ignore'
)

In [27]:
numeric_transformer = StandardScaler()

In [28]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [29]:
logistic_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight = 'balanced',
        max_iter = 1000,
        random_state = RANDOM_STATE
    ))
])

In [30]:
logistic_model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [31]:
y_pred = logistic_model.predict(X_test)
y_proba = logistic_model.predict_proba(X_test)[:, 1]

## Evaluación del modelo

En esta sección se evalúa el desempeño del modelo utilizando métricas adecuadas para un problema de clasificación desbalanceado. Se prioriza el recall de la clase positiva y la capacidad del modelo para rankear clientes según su probabilidad de aceptación.


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90      7308
           1       0.36      0.65      0.46       928

    accuracy                           0.83      8236
   macro avg       0.65      0.75      0.68      8236
weighted avg       0.88      0.83      0.85      8236



In [33]:
confusion_matrix(y_test, y_pred)

array([[6241, 1067],
       [ 329,  599]])

In [34]:
roc_auc = roc_auc_score(y_test, y_proba)
roc_auc

0.8001862773200838

In [35]:
feature_names = (
    logistic_model
    .named_steps['preprocessor']
    .get_feature_names_out()
)

coefficients = logistic_model.named_steps['classifier'].coef_[0]

coef_df = (
    pd.DataFrame({
        'feature' : feature_names,
        'coefficient' : coefficients
    })
    .sort_values(by='coefficient', ascending=False)
)

coef_df.head(10)

,feature,coefficient
41,cat__month_mar,1.488558
5,num__cons.price.idx,1.109203
51,cat__poutcome_success,0.686088
37,cat__month_aug,0.655197
38,cat__month_dec,0.645542
26,cat__education_illiterate,0.406727
7,num__euribor3m,0.403795
8,num__nr.employed,0.382220
22,cat__marital_unknown,0.378458
13,cat__job_retired,0.357646


In [36]:
coef_df.tail(10)

,feature,coefficient
1,num__campaign,-0.113208
46,cat__day_of_week_mon,-0.135875
30,cat__default_unknown,-0.182533
18,cat__job_unemployed,-0.190643
2,num__pdays,-0.236977
43,cat__month_nov,-0.351589
42,cat__month_may,-0.424446
40,cat__month_jun,-0.573730
36,cat__contact_telephone,-0.645599
4,num__emp.var.rate,-2.230914


El modelo muestra que factores temporales como el mes del contacto y el historial previo del cliente tienen un impacto significativo en la aceptación. Además, variables macroeconómicas aportan contexto, mientras que múltiples intentos de contacto y ciertos canales reducen la probabilidad de respuesta.

In [37]:
y_score = logistic_model.predict_proba(X_test)[:, 1]

In [38]:
scored_clients = X_test.copy()
scored_clients['score'] = y_score
scored_clients['actual'] = y_test.values

scored_clients.sort_values(by='score', ascending=False).head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,score,actual
39135,19,student,single,basic.6y,no,yes,no,cellular,mar,tue,...,5,2,success,-1.8,93.369,-34.8,0.655,5008.7,0.984118,1
40368,48,admin.,single,university.degree,no,yes,no,cellular,aug,wed,...,0,2,success,-1.7,94.027,-38.3,0.900,4991.6,0.983365,1
40373,30,blue-collar,single,professional.course,no,no,no,cellular,aug,wed,...,9,1,success,-1.7,94.027,-38.3,0.900,4991.6,0.983126,1
40346,21,student,single,high.school,no,yes,no,cellular,aug,tue,...,9,2,success,-1.7,94.027,-38.3,0.899,4991.6,0.982743,1
39122,66,retired,married,basic.4y,no,yes,no,cellular,mar,tue,...,6,1,success,-1.8,93.369,-34.8,0.655,5008.7,0.982700,1
40105,83,retired,married,university.degree,no,yes,no,cellular,jul,tue,...,6,2,success,-1.7,94.215,-40.3,0.835,4991.6,0.982259,1
40161,26,technician,single,university.degree,no,no,no,cellular,jul,fri,...,6,1,success,-1.7,94.215,-40.3,0.861,4991.6,0.981526,0
39326,27,admin.,single,unknown,no,no,no,cellular,mar,tue,...,3,1,success,-1.8,93.369,-34.8,0.637,5008.7,0.981095,1
40266,24,student,single,professional.course,no,no,no,cellular,jul,wed,...,6,3,success,-1.7,94.215,-40.3,0.896,4991.6,0.980930,1
40442,65,retired,married,professional.course,no,yes,no,cellular,aug,tue,...,6,3,success,-1.7,94.027,-38.3,0.904,4991.6,0.980908,1


In [39]:
threshold = 0.30
y_pred_custom = (y_score >= threshold).astype(int)

In [40]:
top_20 = scored_clients.nlargest(
    int(0.2 * len(scored_clients)),
    'score'
)

In [41]:
top_20['actual'].mean()

np.float64(0.3630843958712811)

## Conclusiones

Se recomienda el uso de la regresión logística como modelo principal, dado que ofrece un buen equilibrio entre desempeño, interpretabilidad y facilidad de implementación en un entorno productivo.


## Limitaciones

El modelo no predice con certeza el comportamiento individual de cada cliente, sino que estima probabilidades basadas en patrones históricos, por lo que su principal valor es la priorización y no la automatización completa de la decisión.
